# Sistema de tipos

Los ingredientes más importantes que caracterizan a Julia y permiten que sea rápido son la **inferencia de tipos** y la **especialización con respecto al tipo**. En conjunto, estos dos ingredientes dan lugar al *despacho múltiple*. Muchos ven esto solo como una característica de Julia, algo que viene con el lenguaje, pero en realidad es un aspecto esencial, con raíces profundas en la base del lenguaje, que permiten que sea rápido.

El andamiaje de Julia es su sistema de tipos. El tipo de un objeto permite interpretar la memoria y saber qué hacer en un llamado de una función. Al nivel de la CPU, todos los objetos tienen un tipo, ya que la CPU necesita saber cómo interpretar los bits. Si no se conocen los tipos, no se puede computar nada. Algunos lenguajes son más explícitos acerca de los tipos, y otros tratan de esconder el tipo del usuario. 

Veamos primero el sistema de tipos de Julia. En Julia, **todos los objetos tienen un tipo** 

In [ ]:
typeof(1.0)

Float64

In [ ]:
typeof(1)

Int64

In [ ]:
typeof("Hola")

String

In [ ]:
typeof(:a)

Symbol

Incluso los tipos son objetos y, por lo tanto, tienen tipo

In [ ]:
typeof(Float64)

DataType

### Relación de subtipo-supertipo

Algunos tipos son subtipo de otros tipos. La relación subitpo-supertipo se designa con el símbolo `<:`

Por ejemplo:

In [ ]:
Int <: Real

true

In [ ]:
Real <: Number

true

Todos los tipos son subtipo de `Any`

In [ ]:
Int <: Any

true

In [ ]:
String <: Any

true

In [ ]:
Vector{Float64} <: Any

true

La relación `<:` satisface:

1. Reflexividad: `A<:A` para todo tipo `A`
2. Antisimetría: si `A<:B` y `B<:A`, entonces `A==B` para todo tipo `A` y `B`
3. Transitividad: si `A<:B` y `B<:C`, entonces `A<:C` para todo tipo `A`, `B` y `C`

Por lo tanto, `<:` es un orden parcial sobre los tipos, con `Any` como máximo. 

Los tipos puede visualizarse en un diagrama de Hasse. El siguiente es el diagrama de los tipos numéricos en la base de Julia:

<div align="center">
  <img src="./img/tipos.png" alt="Tipos numéricos en Julia" width="850"/>
</div>

Crédito de la imagen: [The Julia Express](https://github.com/bkamins/The-Julia-Express)

### Tipos abstractos y concretos

En Julia, los tipos se clasifican en **abstractos** y **concretos**.

Los **tipos abstractos** son solo nodos en el diagrama de tipos. 

No tienen instancias. Es decir, no hay objetos cuyo tipo sea abstracto. Solo sirven para organizar los tipos en una jerarquía.

Los tipos abstractos se declaran con `abstract type`. Por ejemplo, así es la implementación de los tipos numéricos en Julia:

```julia
abstract type Number end

abstract type Real <: Number end

abstract type AbstractFloat <: Real end
abstract type Integer <: Real end

abstract type Signed <: Integer end
abstract type Unsigned <: Integer end
```

También se pueden crear nuevos tipos abstractos:

In [3]:
abstract type Bebida end

abstract type Cafe <: Bebida end
abstract type Te <: Bebida end
abstract type Vino <: Bebida end

Los **tipos concretos** son los que pueden instanciarse y **no admiten subtipos**. 

Los tipos concretos permiten organizar datos. Pueden definirse tipos compuestos con `struct`:

In [13]:
struct Expreso <: Cafe
    marca::String
    precio::Float64
end

Se instancian de la siguiente manera:

In [ ]:
Expreso("Cabrales", 1300.0)

Expreso("Cabrales", 1300.0)

Y, como dijimos, no admiten subtipos:

In [ ]:
struct Ristretto <: Expreso
    marca::String
    precio::Float64
end

LoadError: invalid subtyping in definition of Ristretto: can only subtype abstract types.

Definidos de esta manera, los tipos son inmutables:

In [14]:
a = Expreso("Cabrales", 1300)

a.precio = 1500

LoadError: setfield!: immutable struct of type Expreso cannot be changed

Usando `mutable struct` en lugar de `struct` podemos definir tipos concretos mutables

In [16]:
mutable struct Americano <: Cafe
    marca::String
    precio::Float64
end

a = Americano("Cabrales", 1300)

Americano("Cabrales", 1300.0)

In [18]:
a.precio = 1500
a

Americano("Cabrales", 1500.0)

Por último, también podemos definir tipos concretos sin campos. Por ejemplo,

In [4]:
struct Agua <: Bebida end

Esta clase de tipos se denominan *singletons*, ya que solo tienen una instancia:

In [5]:
agua1 = Agua()
agua2 = Agua()

agua1 === agua2

true

donde `===` es el operador de identidad, que verifica si dos variables refieren al mismo objeto (no si son iguales, sino idénticas). Por ejemplo, el famoso `nothing` de Julia es la instancia *singleton* de `Nothing`.